In [1]:
# import pymysql
# from sqlalchemy import create_engine
# import sys
# from abc import ABC, abstractmethod
# from typing import *

import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 0. 모듈의 상위 패키지를 파이썬 경로로 설정

In [2]:
import sys

In [3]:
sys.path

['/Users/minjoo/codeStudy/psudonymizerTeam/pseudonymizer/tests',
 '/opt/homebrew/anaconda3/lib/python311.zip',
 '/opt/homebrew/anaconda3/lib/python3.11',
 '/opt/homebrew/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/anaconda3/lib/python3.11/site-packages',
 '/opt/homebrew/anaconda3/lib/python3.11/site-packages/aeosa']

In [4]:
# sys.path.append("C:\Pseudonymizer\pseudonymizer")
sys.path.append("/Users/minjoo/codeStudy/psudonymizerTeam/pseudonymizer")
# /path/to/your/module

#### 1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드

In [5]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [6]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "1234")

In [7]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")
SQL = "SELECT * FROM DATA_FINANCE"
queryObject.dataQueryLanguage(sql = SQL)

In [8]:
# DATA_FINANCE_TUPLE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [8]:
DATA_FINANCE_TABLE = queryObject.executeQueryAsDataFrame()

In [9]:
DATA_FINANCE_TABLE

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,...,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION,SERIALNUM
0,0,강현심,female,80,010-1826-4535,27218,충청북도 제천시 수산면 옥순봉로6길 61-32,"판잣집, 비닐하우스",6,441,...,5357418,8583708,3,5,N,0,0,N,0,F1
1,1,서삼봉,female,30,010-1765-1467,12736,경기도 광주시 초월읍 도곡길 109-9,다세대주택,8,515,...,8870904,5328233,3,1,N,0,0,N,0,F2
2,2,팽상아,female,53,010-6593-5091,51292,경상남도 창원시 마산회원구 회원남32길 28-1(회원동),"판잣집, 비닐하우스",1,269,...,8525658,2005663,6,5,N,0,0,N,0,F3
3,3,박옥점,female,26,010-7714-1345,58541,전라남도 무안군 청계면 구로길 65,다가구 단독주택,7,254,...,2593007,8944424,4,1,N,0,0,N,0,F4
4,4,윤경념,female,69,010-5360-6014,31741,충청남도 당진시 신평면 신평길 64-6,영업 겸용 단독주택,8,590,...,411525,5125357,5,3,N,0,0,N,0,F5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,갈재희,female,79,010-9796-1142,36447,경상북도 영덕군 달산면 청련길 10-10,다세대주택,8,868,...,835585,6144642,5,1,N,0,0,Y,0,F94996
94996,94996,림상영,male,53,010-2817-1504,26200,강원특별자치도 영월군 무릉도원면 황정길 253-11,기타,7,875,...,4691445,6000005,5,3,N,0,0,N,0,F94997
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,...,8488084,7377056,8,2,N,0,0,N,0,F94998
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,...,5109440,3625570,3,2,N,0,0,N,0,F94999


In [10]:
type(DATA_FINANCE_TABLE)
DATA_FINANCE_TABLE.info()
DATA_FINANCE_TABLE.tail(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NUM_SERIAL                  95000 non-null  int64 
 1   NAME                        95000 non-null  object
 2   GENDER                      95000 non-null  object
 3   AGE                         95000 non-null  int64 
 4   PHONE_NUMBER                95000 non-null  object
 5   ZIP_CODE                    95000 non-null  int64 
 6   HOME_ADDRESS                95000 non-null  object
 7   HOME_TYPE                   95000 non-null  object
 8   INCOME_BRACKET              95000 non-null  int64 
 9   CREDIT_SCORE                95000 non-null  int64 
 10  REPAYMENT_RISK_INDEX        95000 non-null  int64 
 11  AMT_CREDITCARD_PAYMENT      95000 non-null  int64 
 12  AMT_CASHADVANCE_PAYMENT     95000 non-null  int64 
 13  NUM_CREDITCARD_ISSUANCES    95000 non-null  in

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,...,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION,SERIALNUM
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,...,8488084,7377056,8,2,N,0,0,N,0,F94998
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,...,5109440,3625570,3,2,N,0,0,N,0,F94999
94999,94999,안전웅,male,46,010-9622-1090,10585,경기도 고양시 덕양구 안진6길 12(지축동),오피스텔,7,44,...,874939,5785351,7,0,N,0,0,N,0,F95000


In [11]:
queryObject2 = PyMySQLQuery(pw = "1234")
queryObject2.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")
SQL = "SELECT * FROM DATA_RETAIL"
queryObject2.dataQueryLanguage(sql = SQL)
DATA_RETAIL_TABLE = queryObject2.executeQueryAsDataFrame()

In [12]:
DATA_RETAIL_TABLE

,NUM_SERIAL,NAME,GENDER,AGE,JOIN_DATE,PHONE_NUMBER,ZIP_CODE,SHIPPING_ADDRESS,NUM_PURCHASES_BOOKS,AMT_PURCHASES_BOOKS,NUM_PURCHASES_CULTURE,AMT_PURCHASES_CULTURE,NUM_PURCHASES_EDU,AMT_PURCHASES_EDU,AMT_USAGE_MEMBERSHIP,AMT_USAGE_GIFTCERTIFICATE,SERIALNUM
0,0,맹수원,male,73,2021-02-01,010-7161-5648,26310,강원특별자치도 원주시 소초면 대왕고개길 53-13,949,8588542,858,21911531,7768,1743329,1201564,1569165,R1
1,1,가한성,male,44,2019-12-23,010-4411-3871,48235,부산광역시 수영구 연수로312번길 13(망미동),278,1696132,490,80849464,5430,5331234,1660142,967963,R2
2,2,진노순,female,59,2020-04-15,010-8299-4394,63207,제주특별자치도 제주시 신성로13길 26(이도이동),324,3843932,706,79898363,5974,8858371,3106686,1829878,R3
3,3,성간란,female,34,2020-02-12,010-8178-4332,42146,대구광역시 수성구 희망로36길 122-25(황금동),720,9124604,384,87698526,677,22899412,576470,2423582,R4
4,4,프주희,female,49,2020-01-22,010-9654-3329,32932,충청남도 논산시 강경읍 금백로175번길 23,9,1412817,20,22057893,332,27086237,978040,901883,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,조양립,female,66,2022-09-05,010-3563-1984,17020,경기도 용인시 처인구 관전로 9-3(역북동),233,383644,318,43174511,7581,9938963,2667,136798,R84996
84996,84996,유찬엽,female,28,2022-04-01,010-2837-0382,36407,경상북도 영덕군 영해면 원구길 28-3,139,1365534,270,99210229,9758,46467833,915621,1222556,R84997
84997,84997,김작향,female,36,2022-10-20,010-9707-4542,36929,경상북도 문경시 호계면 막곡길 39-4,351,944905,915,64036013,5488,46471629,103854,249169,R84998
84998,84998,최충상,female,22,2021-03-07,010-4598-7996,32806,충청남도 계룡시 엄사면 번영4길 12-1,578,9401781,477,28015271,3556,41935012,2360528,2680713,R84999


#### 2. 가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환

In [14]:
from pseudonymizer.pseudonymizer import Pseudonymizer

In [15]:
from pseudonymizer.pseudonym import Pseudonym, PseudonymizerModule
    # ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)

In [16]:
help(Pseudonym)

Help on class Pseudonym in module pseudonymizer.pseudonym:

class Pseudonym(builtins.object)
 |  Pseudonym(dataframe, pseudonymizer_module)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe, pseudonymizer_module)
 |      원본정보(재현데이터)와 가명처리 구체 클래스를 인스턴스 변수로 선언하는(초기화) 생성자
 |  
 |  __str__(self)
 |      캡슐화된 데이터셋의 속성(컬럼)정보를 반환하는 메서드
 |  
 |  addDictionary(self, column, pseudonymizers)
 |      가명처리를 수행할 데이터 컬럼명과 해당 열에 적용할 여러 가명처리 기법 리스트를 입력받아 다양한 비식별 조치를 수행할 수 있도록 지정하는 메서드
 |  
 |  addPseudonymizer(self, pseudonymizer)
 |      가명처리 추상 클래스에 대한 자식 클래스를 입력받는 pseudonymizer파라미터를 가지는 메서드
 |  
 |  categorizeEquivalentClass(self, attributes: List[str])
 |      각 행(레코드)에 대한 개인식별가능정보 속성(컬럼)들 사이에 동질 집합을 확인하는 메서드
 |      Pseudonym(dataframe).equivalent_class.keys()를 통해 동질집합 확인
 |  
 |  countEquivalentClass(self)
 |  
 |  getPseudonymizedDataframe(self)
 |      가명처리 데이터 반환
 |  
 |  pseudonymizeData(self)
 |      가명처리 기법을 해당 컬럼에 적용하는 메서드(apply함수를 활용하여 데이터프레임 모든 행, 특정 열에 비식별조치를 취하는 접근방식)
 |

In [13]:
from pseudonymizer.pseudonymizers.maskingPseudonymizer import MaskingPseudonymizer
from pseudonymizer.pseudonymizers.charcategorization import CategorizationOfCharacter
from pseudonymizer.pseudonymizers.numcategorization import CategorizationOfNumeric
from pseudonymizer.pseudonymizers.columncategorization import CategorizationOfColumn
from pseudonymizer.pseudonymizers.microAggregation import MicroAggregation
from pseudonymizer.pseudonymizers.randomRoundingPseudonymizer import RandomRoundingPseudonymizer
from pseudonymizer.pseudonymizers.topandBottomCoding import TopandBottomCoding

In [18]:
# 가명처리 기법 적용 객체 생성
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE,
    pseudonymizer_module = None
)

In [16]:
pseudonym_object2 = Pseudonym(
    dataframe=DATA_RETAIL_TABLE,
    pseudonymizer_module=None
)

In [17]:
pseudonym_object2.categorizeEquivalentClass(
    attributes=["GENDER"]
)

In [21]:
# 원본 데이터의 컬럼 정보 출력(.info())
# print(pseudonym_object)

In [22]:
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object.categorizeEquivalentClass(
    attributes = ["NUM_CREDITCARD_ISSUANCES", "TF_LOAN", "TF_PENSION"]
)

In [23]:
# 동질집합 반환
pseudonym_object.countEquivalentClass()

(1, 'N', 'N') 7951
(1, 'N', 'Y') 596
(1, 'Y', 'N') 840
(1, 'Y', 'Y') 83
(2, 'N', 'N') 7988
(2, 'N', 'Y') 580
(2, 'Y', 'N') 883
(2, 'Y', 'Y') 54
(3, 'N', 'N') 8031
(3, 'N', 'Y') 585
(3, 'Y', 'N') 895
(3, 'Y', 'Y') 69
(4, 'N', 'N') 7960
(4, 'N', 'Y') 627
(4, 'Y', 'N') 902
(4, 'Y', 'Y') 68
(5, 'N', 'N') 7972
(5, 'N', 'Y') 573
(5, 'Y', 'N') 923
(5, 'Y', 'Y') 82
(6, 'N', 'N') 7857
(6, 'N', 'Y') 618
(6, 'Y', 'N') 868
(6, 'Y', 'Y') 71
(7, 'N', 'N') 7887
(7, 'N', 'Y') 593
(7, 'Y', 'N') 878
(7, 'Y', 'Y') 77
(8, 'N', 'N') 8002
(8, 'N', 'Y') 611
(8, 'Y', 'N') 803
(8, 'Y', 'Y') 64
(9, 'N', 'N') 7967
(9, 'N', 'Y') 598
(9, 'Y', 'N') 890
(9, 'Y', 'Y') 50
(10, 'N', 'N') 7899
(10, 'N', 'Y') 639
(10, 'Y', 'N') 906
(10, 'Y', 'Y') 60


In [24]:
pseudonym_object2.addDictionary(
    column="JOIN_DATE",
    pseudonymizers=[age_cate]
)

In [25]:
pseudonym_object2.pseudonymizeData()

In [26]:
pseudonym_object2.getPseudonymizedDataframe()

,NUM_SERIAL,NAME,GENDER,AGE,JOIN_DATE,PHONE_NUMBER,ZIP_CODE,SHIPPING_ADDRESS,NUM_PURCHASES_BOOKS,AMT_PURCHASES_BOOKS,NUM_PURCHASES_CULTURE,AMT_PURCHASES_CULTURE,NUM_PURCHASES_EDU,AMT_PURCHASES_EDU,AMT_USAGE_MEMBERSHIP,AMT_USAGE_GIFTCERTIFICATE,SERIALNUM
0,0,맹수원,male,73,0대 초반,010-7161-5648,26310,강원특별자치도 원주시 소초면 대왕고개길 53-13,949,8588542,858,21911531,7768,1743329,1201564,1569165,R1
1,1,가한성,male,44,0대 초반,010-4411-3871,48235,부산광역시 수영구 연수로312번길 13(망미동),278,1696132,490,80849464,5430,5331234,1660142,967963,R2
2,2,진노순,female,59,0대 초반,010-8299-4394,63207,제주특별자치도 제주시 신성로13길 26(이도이동),324,3843932,706,79898363,5974,8858371,3106686,1829878,R3
3,3,성간란,female,34,0대 초반,010-8178-4332,42146,대구광역시 수성구 희망로36길 122-25(황금동),720,9124604,384,87698526,677,22899412,576470,2423582,R4
4,4,프주희,female,49,0대 초반,010-9654-3329,32932,충청남도 논산시 강경읍 금백로175번길 23,9,1412817,20,22057893,332,27086237,978040,901883,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,조양립,female,66,0대 초반,010-3563-1984,17020,경기도 용인시 처인구 관전로 9-3(역북동),233,383644,318,43174511,7581,9938963,2667,136798,R84996
84996,84996,유찬엽,female,28,0대 초반,010-2837-0382,36407,경상북도 영덕군 영해면 원구길 28-3,139,1365534,270,99210229,9758,46467833,915621,1222556,R84997
84997,84997,김작향,female,36,0대 초반,010-9707-4542,36929,경상북도 문경시 호계면 막곡길 39-4,351,944905,915,64036013,5488,46471629,103854,249169,R84998
84998,84998,최충상,female,22,0대 초반,010-4598-7996,32806,충청남도 계룡시 엄사면 번영4길 12-1,578,9401781,477,28015271,3556,41935012,2360528,2680713,R84999


In [49]:
categorize_col = CategorizationOfColumn(numeric_type="bin", grouping_standard=["A", "B", "C"], right=True, ascending=False)

In [50]:
micro_aggregation = MicroAggregation(calulate_type="average", quasi_identifier=(1, 'N', 'N'))

In [52]:
char_cate = CategorizationOfCharacter(category_type="user_definition", category_mapping={"F": ["female"], "M": ["male"]})

In [23]:
age_cate = CategorizationOfNumeric(numeric_type="age", grouping_standard="5bin")

In [20]:
num_def = CategorizationOfNumeric(numeric_type="user_definition", 
                                  category_mapping={"A": (0, 10000), "B": (10000, 50000), "C": (50000, 100000)})

In [140]:
adad = "sdsd"

In [141]:
# 가명처리 기법 클래스 입력
pseudonym_object.addPseudonymizer(
    pseudonymizer = adad
)

입력받은 sdsd 기술은 가명처리 기법에 추가할 수 없습니다.


In [110]:
# 적용 컬럼과 여러 가명처리 기법 정의
pseudonym_object.addDictionary(
    column = "ZIP_CODE",
    pseudonymizers = [categorize_col]
)

In [135]:
pseudonym_object.addDictionary(
    column = "CREDIT_SCORE",
    pseudonymizers = [micro_aggregation]
)

In [53]:
pseudonym_object.addDictionary(
    column="GENDER",
    pseudonymizers=[char_cate]
)

In [22]:
pseudonym_object.addDictionary(
    column="AGE",
    pseudonymizers=[age_cate]
)

In [23]:
date_char = CategorizationOfCharacter(category_type="date", category_mapping=None)

In [21]:
pseudonym_object2.addDictionary(
    column="ZIP_CODE",
    pseudonymizers=[num_def]
)

In [22]:
pseudonym_object2._pseudonymDictionary

{'ZIP_CODE': [<pseudonymizer.pseudonymizers.numcategorization.CategorizationOfNumeric at 0x300491710>]}

In [23]:
pseudonym_object2.pseudonymizeData()

In [24]:
pseudonym_object2.getPseudonymizedDataframe()

,NUM_SERIAL,NAME,GENDER,AGE,JOIN_DATE,PHONE_NUMBER,ZIP_CODE,SHIPPING_ADDRESS,NUM_PURCHASES_BOOKS,AMT_PURCHASES_BOOKS,NUM_PURCHASES_CULTURE,AMT_PURCHASES_CULTURE,NUM_PURCHASES_EDU,AMT_PURCHASES_EDU,AMT_USAGE_MEMBERSHIP,AMT_USAGE_GIFTCERTIFICATE,SERIALNUM
0,0,맹수원,male,73,2021-02-01,010-7161-5648,B,강원특별자치도 원주시 소초면 대왕고개길 53-13,949,8588542,858,21911531,7768,1743329,1201564,1569165,R1
1,1,가한성,male,44,2019-12-23,010-4411-3871,B,부산광역시 수영구 연수로312번길 13(망미동),278,1696132,490,80849464,5430,5331234,1660142,967963,R2
2,2,진노순,female,59,2020-04-15,010-8299-4394,C,제주특별자치도 제주시 신성로13길 26(이도이동),324,3843932,706,79898363,5974,8858371,3106686,1829878,R3
3,3,성간란,female,34,2020-02-12,010-8178-4332,B,대구광역시 수성구 희망로36길 122-25(황금동),720,9124604,384,87698526,677,22899412,576470,2423582,R4
4,4,프주희,female,49,2020-01-22,010-9654-3329,B,충청남도 논산시 강경읍 금백로175번길 23,9,1412817,20,22057893,332,27086237,978040,901883,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,조양립,female,66,2022-09-05,010-3563-1984,B,경기도 용인시 처인구 관전로 9-3(역북동),233,383644,318,43174511,7581,9938963,2667,136798,R84996
84996,84996,유찬엽,female,28,2022-04-01,010-2837-0382,B,경상북도 영덕군 영해면 원구길 28-3,139,1365534,270,99210229,9758,46467833,915621,1222556,R84997
84997,84997,김작향,female,36,2022-10-20,010-9707-4542,B,경상북도 문경시 호계면 막곡길 39-4,351,944905,915,64036013,5488,46471629,103854,249169,R84998
84998,84998,최충상,female,22,2021-03-07,010-4598-7996,B,충청남도 계룡시 엄사면 번영4길 12-1,578,9401781,477,28015271,3556,41935012,2360528,2680713,R84999


In [25]:
DATA_RETAIL_TABLE

,NUM_SERIAL,NAME,GENDER,AGE,JOIN_DATE,PHONE_NUMBER,ZIP_CODE,SHIPPING_ADDRESS,NUM_PURCHASES_BOOKS,AMT_PURCHASES_BOOKS,NUM_PURCHASES_CULTURE,AMT_PURCHASES_CULTURE,NUM_PURCHASES_EDU,AMT_PURCHASES_EDU,AMT_USAGE_MEMBERSHIP,AMT_USAGE_GIFTCERTIFICATE,SERIALNUM
0,0,맹수원,male,73,2021-02-01,010-7161-5648,26310,강원특별자치도 원주시 소초면 대왕고개길 53-13,949,8588542,858,21911531,7768,1743329,1201564,1569165,R1
1,1,가한성,male,44,2019-12-23,010-4411-3871,48235,부산광역시 수영구 연수로312번길 13(망미동),278,1696132,490,80849464,5430,5331234,1660142,967963,R2
2,2,진노순,female,59,2020-04-15,010-8299-4394,63207,제주특별자치도 제주시 신성로13길 26(이도이동),324,3843932,706,79898363,5974,8858371,3106686,1829878,R3
3,3,성간란,female,34,2020-02-12,010-8178-4332,42146,대구광역시 수성구 희망로36길 122-25(황금동),720,9124604,384,87698526,677,22899412,576470,2423582,R4
4,4,프주희,female,49,2020-01-22,010-9654-3329,32932,충청남도 논산시 강경읍 금백로175번길 23,9,1412817,20,22057893,332,27086237,978040,901883,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,조양립,female,66,2022-09-05,010-3563-1984,17020,경기도 용인시 처인구 관전로 9-3(역북동),233,383644,318,43174511,7581,9938963,2667,136798,R84996
84996,84996,유찬엽,female,28,2022-04-01,010-2837-0382,36407,경상북도 영덕군 영해면 원구길 28-3,139,1365534,270,99210229,9758,46467833,915621,1222556,R84997
84997,84997,김작향,female,36,2022-10-20,010-9707-4542,36929,경상북도 문경시 호계면 막곡길 39-4,351,944905,915,64036013,5488,46471629,103854,249169,R84998
84998,84998,최충상,female,22,2021-03-07,010-4598-7996,32806,충청남도 계룡시 엄사면 번영4길 12-1,578,9401781,477,28015271,3556,41935012,2360528,2680713,R84999


In [73]:
pseudonym_object2.addDictionary(
    column="JOIN_DATE",
    pseudonymizers=[date_char]
)

In [144]:
pseudonym_object._pseudonymizers

[]

In [24]:
pseudonym_object._pseudonymDictionary

{'AGE': [<pseudonymizer.pseudonymizers.numcategorization.CategorizationOfNumeric at 0x17dd91c10>]}

In [25]:
# 가명처리 수행
pseudonym_object.pseudonymizeData()

AttributeError: 'int' object has no attribute 'month'

In [56]:
# 가명정보 반환
pseudonym_data = pseudonym_object.getPseudonymizedDataframe()

In [57]:
pseudonym_data

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,...,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION,SERIALNUM
0,0,강현심,F,80,010-1826-4535,27218,충청북도 제천시 수산면 옥순봉로6길 61-32,"판잣집, 비닐하우스",6,441,...,5357418,8583708,3,5,N,0,0,N,0,F1
1,1,서삼봉,F,30,010-1765-1467,12736,경기도 광주시 초월읍 도곡길 109-9,다세대주택,8,515,...,8870904,5328233,3,1,N,0,0,N,0,F2
2,2,팽상아,F,53,010-6593-5091,51292,경상남도 창원시 마산회원구 회원남32길 28-1(회원동),"판잣집, 비닐하우스",1,269,...,8525658,2005663,6,5,N,0,0,N,0,F3
3,3,박옥점,F,26,010-7714-1345,58541,전라남도 무안군 청계면 구로길 65,다가구 단독주택,7,254,...,2593007,8944424,4,1,N,0,0,N,0,F4
4,4,윤경념,F,69,010-5360-6014,31741,충청남도 당진시 신평면 신평길 64-6,영업 겸용 단독주택,8,590,...,411525,5125357,5,3,N,0,0,N,0,F5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,갈재희,F,79,010-9796-1142,36447,경상북도 영덕군 달산면 청련길 10-10,다세대주택,8,868,...,835585,6144642,5,1,N,0,0,Y,0,F94996
94996,94996,림상영,M,53,010-2817-1504,26200,강원특별자치도 영월군 무릉도원면 황정길 253-11,기타,7,875,...,4691445,6000005,5,3,N,0,0,N,0,F94997
94997,94997,명시라,F,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,...,8488084,7377056,8,2,N,0,0,N,0,F94998
94998,94998,롤미라,F,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,...,5109440,3625570,3,2,N,0,0,N,0,F94999


In [33]:
from pseudonymizer.pseudonymizer import Pseudonymizer
from datetime import datetime

class CategorizationOfNumeric(Pseudonymizer):
    """수치(연속형) 데이터를 임의의 수를 기준으로 범위(범주형)으로 설정하는 가명처리기법 구체클래스 """
    def __init__(self, numeric_type, grouping_standard=None, category_mapping: dict = None):
        self.numeric_type = numeric_type
        self.grouping_standard = grouping_standard
        self.category_mapping = category_mapping
    
    def pseudonymizeData(self, input_numeric: float):
        """식별성이 높은 그룹을 하나로 묶는 메서드"""
        if self.numeric_type == "age":
            return self.pseudonymizeAge(input_numeric, self.grouping_standard)
        elif self.numeric_type == "income":
            return self.pseudonymizeIncome(input_numeric, self.grouping_standard)      
        elif self.numeric_type == "user_definition":
            return self.pseudonymizeDefinition(input_numeric, self.category_mapping)
        else: 
            raise ValueError(f"{self.numeric_type}은 유효한 범주화 기법 적용 유형이 아닙니다.")
    
    def pseudonymizeAge(self, birthday, grouping_standard):
        """연령 범주화 메서드
        일반적으로 나이 + 주소 + 성별 조합(동질 집합)이 재식별 가능성 있으므로
        5세, 10세 단위 또는 초중후반으로 만 나이 범주화"""
        currentdate = datetime.now().date()
        if (currentdate.month, currentdate.day) < (birthday.month, birthday.day):
        # 아직 현재 날짜가 생일 전인 경우 만 나이 계산 시 한 살 제외
            age = currentdate.year - birthday.year - 1
        else:
            age = currentdate.year - birthday.year
        
        if grouping_standard in ["3bin", "5bin", "10bin"]:
            if grouping_standard == "3bin":
            # 0,1,2(초반) / 3,4,5,6(중반) / 7,8,9(후반)
                sort = (age % 10) // 3
                range = (age // 10) * 10
                if sort == 0:
                    return f"{range}대 초반"
                elif sort == 1:
                    return f"{range}대 중반"
                elif sort == 2:
                    return f"{range}대 후반"
                else: 
                    return
            elif grouping_standard == "5bin":
            # 0,1,2,3,4(초반) / 5,6,7,8,9(후반)
                return f"{(age // 10) * 10}대 초반" if (age % 10) < 5 else f"{(age // 10) * 10}대 후반"
            elif grouping_standard == "10bin":
            # 10대~100대
                return f"{(age // 10) * 10}대"
            else:
                raise ValueError("입력받은 {}은 연령 범주화 기준으로 유효하지 않습니다.".format(grouping_standard))
        
    def pseudonymizeIncome(self, income, grouping_standard):
        """소득금액 범주화 메서드
        소득을 전체 대상자를 9분위(2024년 건강보험료 1인 기준 소득분위)로 균등 분할"""
        if grouping_standard is None:
            threshold_list= [1841500, 2025500, 2675000, 2897000, 3120000, 3343000, 3566000, 3789000, 4012000]
            for index, threshold in enumerate(threshold_list, start = 1):
                if income <= threshold:
                    return f"{index}분위"
        else:
        # While grouping_standard True:
            grouping_standard.sort()
            # 오름차순 정렬 시 일반적으로 사용하는 버블 정렬은 O(N**2)이므로 
            # 시간복잡도 낮추려면 병합 정렬 O(NlogN) 활용 -> 추후 merge_sort() 메서드 적용하여 리팩토링
            for index, grouping_standard in enumerate(grouping_standard, start = 1):
                if income <= grouping_standard:
                    return f"{index}분위"
            
    def pseudonymizeDefinition(self, numeric_tobeclassified, category_mapping: dict):
        """사용자가 직접 구간을 설정하도록 하는 범주화 메서드
        intervals: [(0, 1000), (1000, 5000), (5000, 10000)]
        """
        for category, interval in category_mapping.items():
            if not isinstance(interval, tuple) or len(interval) != 2:
                return f"유효하지 않은 구간 {interval}입니다."
            
            lower, upper = interval # tuple(lower, upper)
            print(lower, upper)
            if lower <= numeric_tobeclassified <= upper:
                return category
        return "other types"

In [34]:
num_def = CategorizationOfNumeric(numeric_type="user_definition", 
                                  category_mapping={"A": (0, 10000), "B": (10000, 50000), "C": (50000, 100000)})

In [35]:
pseudonym_object2.addDictionary(
    column="ZIP_CODE",
    pseudonymizers=[num_def]
)

In [36]:
pseudonym_object2._pseudonymDictionary

{'ZIP_CODE': [<__main__.CategorizationOfNumeric at 0x3023dfb50>]}

In [37]:
pseudonym_object2.pseudonymizeData()

0 10000


TypeError: '<=' not supported between instances of 'int' and 'str'

In [35]:
pseudonym_object2.getPseudonymizedDataframe()

,NUM_SERIAL,NAME,GENDER,AGE,JOIN_DATE,PHONE_NUMBER,ZIP_CODE,SHIPPING_ADDRESS,NUM_PURCHASES_BOOKS,AMT_PURCHASES_BOOKS,NUM_PURCHASES_CULTURE,AMT_PURCHASES_CULTURE,NUM_PURCHASES_EDU,AMT_PURCHASES_EDU,AMT_USAGE_MEMBERSHIP,AMT_USAGE_GIFTCERTIFICATE,SERIALNUM
0,0,맹수원,male,73,2021-02-01,010-7161-5648,"유효하지 않은 구간 [(0, 10000)]입니다.",강원특별자치도 원주시 소초면 대왕고개길 53-13,949,8588542,858,21911531,7768,1743329,1201564,1569165,R1
1,1,가한성,male,44,2019-12-23,010-4411-3871,"유효하지 않은 구간 [(0, 10000)]입니다.",부산광역시 수영구 연수로312번길 13(망미동),278,1696132,490,80849464,5430,5331234,1660142,967963,R2
2,2,진노순,female,59,2020-04-15,010-8299-4394,"유효하지 않은 구간 [(0, 10000)]입니다.",제주특별자치도 제주시 신성로13길 26(이도이동),324,3843932,706,79898363,5974,8858371,3106686,1829878,R3
3,3,성간란,female,34,2020-02-12,010-8178-4332,"유효하지 않은 구간 [(0, 10000)]입니다.",대구광역시 수성구 희망로36길 122-25(황금동),720,9124604,384,87698526,677,22899412,576470,2423582,R4
4,4,프주희,female,49,2020-01-22,010-9654-3329,"유효하지 않은 구간 [(0, 10000)]입니다.",충청남도 논산시 강경읍 금백로175번길 23,9,1412817,20,22057893,332,27086237,978040,901883,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,조양립,female,66,2022-09-05,010-3563-1984,"유효하지 않은 구간 [(0, 10000)]입니다.",경기도 용인시 처인구 관전로 9-3(역북동),233,383644,318,43174511,7581,9938963,2667,136798,R84996
84996,84996,유찬엽,female,28,2022-04-01,010-2837-0382,"유효하지 않은 구간 [(0, 10000)]입니다.",경상북도 영덕군 영해면 원구길 28-3,139,1365534,270,99210229,9758,46467833,915621,1222556,R84997
84997,84997,김작향,female,36,2022-10-20,010-9707-4542,"유효하지 않은 구간 [(0, 10000)]입니다.",경상북도 문경시 호계면 막곡길 39-4,351,944905,915,64036013,5488,46471629,103854,249169,R84998
84998,84998,최충상,female,22,2021-03-07,010-4598-7996,"유효하지 않은 구간 [(0, 10000)]입니다.",충청남도 계룡시 엄사면 번영4길 12-1,578,9401781,477,28015271,3556,41935012,2360528,2680713,R84999


#### 3. 가명처리 절차 : 가명정보에 프라이버시 보호 모델 적용

In [25]:
from pseudonymizer.privacyPreservingModels import PrivacyPreservingModel

In [27]:
help(PrivacyPreservingModel)

Help on class PrivacyPreservingModel in module pseudonymizer.privacyPreservingModels:

class PrivacyPreservingModel(builtins.object)
 |  PrivacyPreservingModel(dataframe, epsilon)
 |  
 |  개인식별가능정보 속성을 기준으로 그룹화된 데이터로 프라이버시 보호 모델을 적용하여 정량적인 위험성을 규정하는 실행 클래스
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe, epsilon)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      동질집합에 대한 정보를 문자열로 반환하는 메서드
 |  
 |  applyDPrivacy(self, method, boundary, attributes, sensitive_attribute, outlier)
 |      차분 프라이버시 기법(차등적 정보보호 기능)을 적용하는 메서드
 |  
 |  applyKAnonymityOrLDiversity(self, method: str, K: int, L: int, attributes: List[str], sensitive_attribute)
 |      K-익명성과 L-다양성 모델을 선택적으로 적용하는 메서드
 |      input
 |      -----
 |      method: 프라이버시 보호 모델 메서드를 받고, 
 |      keyword arguments에 딕셔너리 형식으로 각 기법에 필요한 파라미터를 받아옴
 |  
 |  applyLocalLDiversity(self, K: int, L: int, attributes: List[str], sensitive_attribute: str, LocalL: int)
 |  
 |  apply